In [1]:
import heapq
import math
import random
from typing import List, Union, Optional, Set, Tuple


In [2]:
matrix = [
    # Actores                       Tomas
    #1  2  3  4  5  6  7  8  9 10
    [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],  # 01
    [0, 0, 1, 1, 1, 0, 0, 0, 0, 0],  # 02
    [0, 1, 0, 0, 1, 0, 1, 0, 0, 0],  # 03
    [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],  # 04
    [0, 1, 0, 1, 0, 0, 0, 1, 0, 0],  # 05
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],  # 06
    [1, 1, 0, 1, 1, 0, 0, 0, 0, 0],  # 07
    [1, 1, 0, 0, 0, 1, 0, 0, 0, 0],  # 08
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 0],  # 09
    [1, 1, 0, 0, 0, 1, 0, 0, 1, 0],  # 10
    [1, 1, 1, 0, 1, 0, 0, 1, 0, 0],  # 11
    [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],  # 12
    [1, 0, 0, 1, 1, 0, 0, 0, 0, 0],  # 13
    [1, 0, 1, 0, 0, 1, 0, 0, 0, 0],  # 14
    [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],  # 15
    [0, 0, 0, 1, 0, 0, 0, 0, 0, 1],  # 16
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # 17
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],  # 18
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # 19
    [1, 0, 1, 1, 1, 0, 0, 0, 0, 0],  # 20
    [0, 0, 0, 0, 0, 1, 0, 1, 0, 0],  # 21
    [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],  # 22
    [1, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # 23
    [0, 0, 1, 0, 0, 1, 0, 0, 0, 0],  # 24
    [1, 1, 0, 1, 0, 0, 0, 0, 0, 1],  # 25
    [1, 0, 1, 0, 1, 0, 0, 0, 1, 0],  # 26
    [0, 0, 0, 1, 1, 0, 0, 0, 0, 0],  # 27
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # 28
    [1, 0, 0, 0, 1, 1, 0, 0, 0, 0],  # 29
    [1, 0, 0, 1, 0, 0, 0, 0, 0, 0],  # 30
]
outputs = list()

In [3]:
SolutionType = List[int]
NumericType = Union[float, int]
ActivityType = Union[int, bool]
MatrixType = List[List[ActivityType]]

In [4]:
takes = [
    {i for i, val in enumerate(matrix[j]) if val == 1}
    for j in range(len(matrix))
]

In [41]:
def costo_total(solution: SolutionType, takes_sets: List[Set[int]]) -> int:
    actors_per_day = [set() for _ in range(max(solution) + 1)]
    for take_id, day_id in enumerate(solution):
        actors_per_day[day_id].update(takes_sets[take_id])
    return sum(len(s) for s in actors_per_day)

def normalizar_solucion(solucion):
    dias_usados = sorted(list(set(solucion)))
    mapa_dias = {dia_viejo: nuevo_indice for nuevo_indice, dia_viejo in enumerate(dias_usados)}
    solucion_normalizada = [mapa_dias[dia] for dia in solucion]
    return solucion_normalizada

def probabilidad(T: float, d: float) -> bool:
    """
    Función de probabilidad para aceptar peores soluciones
    :param T: Temperatura
    :param d:
    :return:
    """
    if d < 0: return True
    if T <= 1e-9: return False
    return random.random() < math.exp(-d / T)

def bajar_temperatura_lundy_mees(T: float, beta: float = 0.0001) -> float:
    """
    Función de descenso de temperatura basado en la ecuación de Lundy-Mess
    :param T: Temperatura
    :param beta: coeficiente
    :return:
    """
    return T / (1 + beta * T)


def generate_greedy_solution(max_tomas_por_dia: int = 6):
    takes = []
    for i in range(len(matrix)):
        heapq.heappush(takes, (-sum(matrix[i]), i))
    k = 0
    solution = [0] * len(matrix)
    while takes:
        _, i = heapq.heappop(takes)
        solution[i] = k // max_tomas_por_dia
        k+=1
    return solution

def generar_vecina(solucion: SolutionType, max_tomas: int = 6) -> SolutionType:
    vecino = list(solucion) #copy
    num_tomas = len(vecino)
    if random.random() > 0.5: #swap
        for _ in range(10):
            t1 = random.randint(0, num_tomas-1)
            t2 = random.randint(0, num_tomas-1)
            if vecino[t1] != vecino[t2]:
                vecino[t1], vecino[t2] = vecino[t2], vecino[t1]
                return normalizar_solucion(vecino)
    else:  # mover una toma a otro dia valido
        for _ in range(10):
            toma = random.randint(0, num_tomas-1)
            dia_origen = vecino[toma]
            max_dia = max(vecino)
            dia_destino = random.randint(0, max_dia+1)
            if dia_origen != dia_destino:
                if vecino.count(dia_destino) < max_tomas:
                    vecino[toma] = dia_destino
                    return normalizar_solucion(vecino)
    return vecino



In [36]:
def recocido_simulado(
        solucion_inicial: Optional[SolutionType],
        TEMPERATURA: float,
        min_temp: float = 0.0001
) -> Tuple[SolutionType, NumericType]:
    if solucion_inicial:
        solucion_referencia = list(solucion_inicial)
    else:
        solucion_referencia = random.sample(list(range(30)), 30)
        # solucion_referencia = generate_greedy_solution(6)
    distancia_referencia = costo_total(solucion_referencia, takes)

    mejor_solucion = list(solucion_referencia)  #x* del pseudocodigo
    mejor_distancia = distancia_referencia  #F* del pseudocodigo

    N = 0

    while TEMPERATURA > min_temp: # no converge
        N += 1
        #Genera una solución vecina
        vecina = generar_vecina(solucion_referencia, 6)
        costo_vecina = costo_total(vecina, takes)
        delta = costo_vecina - distancia_referencia
        #Si la nueva vecina es mejor se cambia
        #Si es peor se cambia según una probabilidad que depende de T y delta
        # (distancia_referencia - distancia_vecina)
        if delta < 0 or probabilidad(TEMPERATURA, delta):
            solucion_referencia = list(vecina)
            distancia_referencia = costo_vecina

            #Si es la mejor solución de todas se guarda(siempre!!!)
            if distancia_referencia < mejor_distancia:
                mejor_solucion = list(vecina)
                mejor_distancia = distancia_referencia

        #Bajamos la temperatura
        TEMPERATURA = bajar_temperatura_lundy_mees(TEMPERATURA)

    print("La mejor solución encontrada es ", end="")
    print(mejor_solucion, mejor_distancia)
    return mejor_solucion, mejor_distancia


In [44]:
sol, dist = recocido_simulado(None, 100, min_temp=0.025)

La mejor solución encontrada es [1, 2, 0, 0, 0, 4, 4, 1, 4, 1, 0, 1, 2, 3, 0, 4, 2, 3, 3, 2, 3, 0, 3, 3, 4, 1, 4, 2, 1, 2] 27


In [8]:
def resolver_sa_multi_start(temp=100, min_temp: float = 0.0001, intentos=3, ):
    mejor_solucion_global = None
    mejor_costo_global = float('inf')

    for i in range(intentos):

        semilla = random.sample(list(range(30)), 30)

        solucion_candidata, costo_candidato = recocido_simulado(
            solucion_inicial=semilla,
            TEMPERATURA=temp,
            min_temp=min_temp
        )

        # 3. SELECCIÓN: Nos quedamos con el mejor de todos los mundos
        if costo_candidato < mejor_costo_global:
            mejor_costo_global = costo_candidato
            mejor_solucion_global = solucion_candidata

    return mejor_solucion_global, mejor_costo_global

In [11]:
sol, dist = resolver_sa_multi_start(1000, min_temp=0.0125, intentos=3)

La mejor solución encontrada es [0, 4, 0, 0, 1, 1, 0, 3, 3, 3, 0, 3, 4, 2, 0, 1, 2, 2, 4, 4, 1, 3, 3, 2, 1, 2, 4, 1, 2, 4] 29
La mejor solución encontrada es [3, 3, 0, 0, 1, 0, 3, 2, 1, 2, 1, 1, 1, 2, 0, 0, 4, 4, 4, 2, 1, 3, 4, 4, 0, 2, 3, 3, 2, 4] 30
La mejor solución encontrada es [0, 4, 1, 1, 3, 0, 0, 3, 3, 2, 1, 2, 4, 2, 1, 0, 2, 3, 4, 4, 3, 2, 3, 1, 0, 2, 4, 4, 1, 0] 30
